IMPORT LIBRARIES

In [59]:
import pandas as pd
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn import linear_model 
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression,Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor,GradientBoostingRegressor
import xgboost as xgb
from sklearn.svm import SVR,SVC


# LOAD AND SHOW DATA

In [ ]:
data = pd.read_csv("https://raw.githubusercontent.com/Sumit-saha12/Shipment-price-prediction/main/SCMS_Delivery_History_Dataset.csv")
data.head()

## OVERVIEW DATA

In [ ]:
print('Total number of rows: ',data.shape[0])
print('Total number of column :',data.shape[1])
data.dtypes

## DROP COLUMNS

In [ ]:
data=data.drop(['Vendor','Item Description','Molecule/Test Type'],axis=1)
data.head()

## DATA CLEAN

In [ ]:
data=data[data['First Line Designation']!='No']
data=data[data['Weight (Kilograms)']!='Weight Captured Separately']
data=data[data['Freight Cost (USD)']!='Freight Included in Commodity Cost']
data=data[data['Freight Cost (USD)']!='Invoiced Separately']
data=data[data['Freight Cost (USD)']!='Freight Included in Commodity Cost']
data.head()


Convert import datas object to numerics

In [103]:
data['Weight (Kilograms)'] = pd.to_numeric(data['Weight (Kilograms)'])
data['Freight Cost (USD)'] = pd.to_numeric(data['Freight Cost (USD)'])

data.dtypes

ID                                int64
Project Code                     object
PQ #                             object
PO / SO #                        object
ASN/DN #                         object
Country                          object
Managed By                       object
Fulfill Via                      object
Vendor INCO Term                 object
Shipment Mode                    object
PQ First Sent to Client Date     object
PO Sent to Vendor Date           object
Scheduled Delivery Date          object
Delivered to Client Date         object
Delivery Recorded Date           object
Product Group                    object
Sub Classification               object
Brand                            object
Dosage                           object
Dosage Form                      object
Unit of Measure (Per Pack)        int64
Line Item Quantity                int64
Line Item Value                 float64
Pack Price                      float64
Unit Price                      float64


In [ ]:
data.describe()

## ANALYSIS OF DATA

Creating data for analysis

In [ ]:
order=data['PO / SO #'].nunique()
total_order=data['PO / SO #'].count()
duplicate_orders=total_order-order
total_unique_projects=data['Project Code'].nunique()
total_Freight_Cost=data['Freight Cost (USD)'].sum()
total_weight=data['Weight (Kilograms)'].sum()
avg_freight_cost_uniqueorder=total_Freight_Cost/order
avg_weight_uniqueorder=total_weight/order
unique_country=data['Country'].nunique()
unique_shipment_mode=data['Shipment Mode'].nunique()
country_summary=data.groupby(['Shipment Mode','Country','Freight Cost (USD)','Weight (Kilograms)']).sum().reset_index()
country_summary=country_summary[['Shipment Mode','Country','Freight Cost (USD)','Weight (Kilograms)']]
country_summary=data.groupby(['Shipment Mode','Country']).agg({'PO / SO #':'count','Freight Cost (USD)':'sum','Weight (Kilograms)':'sum'}).reset_index()
country_summary['freight_cost_by_order']=country_summary['Freight Cost (USD)']/country_summary['PO / SO #']
top_ten_country_by_freight_cost=data.groupby(['Country'])['Freight Cost (USD)'].sum().nlargest(10).reset_index()
mf_summary=data.groupby(['Manufacturing Site']).agg({'PO / SO #':'count','Freight Cost (USD)':'sum'}).reset_index()


Summary of some data

In [ ]:
country_summary.head()

In [ ]:
top_ten_country_by_freight_cost.head()

In [ ]:
mf_summary.head()

## DATA VISUALIZATION

DATA HISTOGRAM GRAPH

In [ ]:
data.hist(figsize=(20,15))

Co-relation attributes each other

In [ ]:
attributes = ['Unit Price','Pack Price','Line Item Quantity','Unit of Measure (Per Pack)','Freight Cost (USD)','Weight (Kilograms)']
scatter_matrix(data[attributes],figsize=(20,15))

In [ ]:
plt.boxplot(data['Weight (Kilograms)'])

In [ ]:
data.boxplot(figsize=(20,15))


Freight_cost_by_order_by_country_order

In [ ]:

plt.figure(figsize=(30,10))
plt.rc({'font.size':30})
plt.bar(country_summary.Country,country_summary.freight_cost_by_order)
plt.xticks(rotation='vertical')
plt.ylabel('freight_cost_by_order')
plt.xlabel('Country')
plt.title('freight_cost_by_order_by_country_order')
plt.show()

Freight_cost_by_order_by_shipment_mode

In [ ]:
plt.figure(figsize=(30,10))
plt.rc({'font.size':30})
# plt.bar(country_summary.freight_cost_by_order,country_summary.Shipment Mode)
sns.barplot(x='Shipment Mode',y='freight_cost_by_order',data=country_summary)
plt.ylabel('freight_cost_by_order')
plt.xlabel('Shipment Mode')
plt.title('freight_cost_by_order_by_shipment_mode')
plt.show()

Order_by_country

In [ ]:
plt.figure(figsize=(30,10))
plt.rc({'font.size':30})
sns.barplot(x='Country',y='PO / SO #',data=country_summary)
plt.xticks(rotation='vertical')
plt.ylabel('orders')
plt.xlabel('Country')
plt.title('order_by_country')
plt.show()

This graph is displaying that Vietnam is a highest order country

Top_ten_country_by_freight_cost

In [ ]:
plt.figure(figsize=(40,20))
plt.rc({'font.size':30})
sns.barplot(x='Country',y='Freight Cost (USD)',data=top_ten_country_by_freight_cost)
plt.xticks(rotation='vertical')
plt.ylabel('Freight Cost (USD)')
plt.xlabel('Country')
plt.title('Top_ten_country_by_freight_cost')
plt.show()

Here Nigeria is a top country by freight cost

Manufacturing Site by freight cost

In [ ]:
plt.figure(figsize=(80,40))
plt.rc({'font.size':120})
sns.barplot(x='Manufacturing Site',y='Freight Cost (USD)',data=mf_summary)
plt.xticks(rotation='vertical')
plt.ylabel('Freight Cost (USD)')
plt.xlabel('Manufacturing Site ')
plt.title('Manufacturing Site by freight cost')
plt.show()

Manufacturing Site by order

In [ ]:
plt.figure(figsize=(40,20))
plt.rc({'font.size':30})
sns.barplot(x='Manufacturing Site',y='PO / SO #',data=mf_summary)
plt.xticks(rotation='vertical')
plt.ylabel('Manufacturing Site ')
plt.xlabel('Order')
plt.title('Manufacturing Site by order')
plt.show()

The above graph is displaying that Aurobindo Unit III,India is highest manufacturing site.

UN NECESSARY DATA REMOVE

In [80]:
X=data.drop(['Pack Price','Unit Price','Freight Cost (USD)','First Line Designation','Line Item Insurance (USD)','Manufacturing Site','Dosage Form','Dosage','Brand','Sub Classification','Product Group','Delivery Recorded Date','Delivered to Client Date','Scheduled Delivery Date','PO Sent to Vendor Date','PQ First Sent to Client Date','Shipment Mode','Vendor INCO Term','Fulfill Via','Managed By','Country','ASN/DN #','PO / SO #','PQ #','Project Code'],axis=1)
data_1=X.copy()
Y1=data['Pack Price']
Y2= data['Unit Price']
Y3=data['Freight Cost (USD)']
X.head(10)
# data_1.head(10)

,ID,Unit of Measure (Per Pack),Line Item Quantity,Line Item Value,Weight (Kilograms),Freight Cost (USD)
0,1,30,19,551.00,13,780.34
1,3,240,1000,6200.00,358,4521.50
2,4,100,500,40000.00,171,1653.78
3,15,60,31920,127360.80,1855,16007.06
4,16,60,38000,121600.00,7590,45450.08
5,23,240,416,2225.60,504,5920.42
7,45,60,16667,60834.55,1478,6212.41
11,61,20,2500,100000.00,853,13569.49
13,64,60,10000,99800.00,7416,64179.42
14,65,100,750,60000.00,256,1760.32


DATA SPLITTING

In [81]:
X_train,X_test = train_test_split(X,test_size=0.2,random_state=0)
Y1_train,Y1_test = train_test_split(Y1,test_size=0.2,random_state=0)
Y2_train,Y2_test = train_test_split(Y2,test_size=0.2,random_state=0)
Y3_train,Y3_test = train_test_split(Y3,test_size=0.2,random_state=0)

In [82]:
y1_mean=Y1_test.mean()
y1_mean

25.417910931174088

## FOR PACK PRICE

MODEL FITTING

In [83]:
final_list=[]
final_list2=[]
final_list3=[]
model=[LinearRegression(),DecisionTreeRegressor(),RandomForestRegressor(),xgb.XGBRegressor(),SVR(),GradientBoostingRegressor(),Lasso()]
result={}
for j in [5, 10, 15, 20,25, 30,35,39]:
        new=[]
       
        for i in model:
            
            
            i.fit(X_train,Y1_train)
            pred=i.predict(X_test)
            rmse = np.sqrt(mean_squared_error(Y1_test, pred))
            new.append(i.score(X_test, Y1_test))
            # scores=(cross_val_score(i,X_train,Y1_train,scoring="neg_mean_squared_error"))
        final_list.append(new)
        # final_list2.append(scores)
        # final_list3.append(r2_score(sc)ores))

In [84]:
for i in model:
    scores=(cross_val_score(i,X_train,Y1_train,scoring="neg_mean_squared_error"))
    print(scores.mean(),end="\t")

-1705.8394082665357	-597.4557822064778	-487.90526650601214	-368.84509329854956	-2005.9405946073603	-340.6204267597239	-1705.8342172670805	

SHOW R2_SCORE VALUES IN TABLE

In [85]:
pd.DataFrame(final_list,index=['5', '10', '15', '20', '25', '30','35','39'], columns=['Linear Regression','Decision Tree Regressor','Random Forest Regressor','XGBRegressor()','svr','gradient boost','lasso'])

,Linear Regression,Decision Tree Regressor,Random Forest Regressor,XGBRegressor(),svr,gradient boost,lasso
5,0.05844,0.668169,0.591322,0.679592,-0.026428,0.717638,0.058449
10,0.05844,0.666778,0.599448,0.679592,-0.026428,0.723765,0.058449
15,0.05844,0.678349,0.611496,0.679592,-0.026428,0.650672,0.058449
20,0.05844,0.630018,0.605863,0.679592,-0.026428,0.662010,0.058449
25,0.05844,0.638601,0.585777,0.679592,-0.026428,0.712599,0.058449
30,0.05844,0.656929,0.628185,0.679592,-0.026428,0.664705,0.058449
35,0.05844,0.666480,0.587063,0.679592,-0.026428,0.690539,0.058449
39,0.05844,0.663372,0.617830,0.679592,-0.026428,0.740286,0.058449


SHOW TABLE FOR EVERY REGRESSOR ERROR VALUE 

In [86]:
all=[]
mse=[]
rms=[]
r2score=[]

for i in model:
    i.fit(X_train,Y1_train)
    pred=i.predict(X_test)
    mse.append(mean_squared_error(Y1_test, pred))
    rms.append(np.sqrt(mean_squared_error(Y1_test, pred)))
    r2score.append(r2_score(Y1_test, pred))
all.append(mse)
all.append(rms)
all.append(r2score)

In [87]:
pd.DataFrame(all,index=['MEAN SQUARED ERROR','ROOT MEAN SQUARED ERROR','R2 SCORE'], columns=['Linear Regression','Decision Tree Regressor','Random Forest Regressor','XGBRegressor','svr','gradient boost','Lasso'])

,Linear Regression,Decision Tree Regressor,Random Forest Regressor,XGBRegressor,svr,gradient boost,Lasso
MEAN SQUARED ERROR,3502.278516,1202.558733,1492.269530,1191.808114,3817.956952,1147.820922,3502.243690
ROOT MEAN SQUARED ERROR,59.180052,34.677929,38.629905,34.522574,61.789618,33.879506,59.179757
R2 SCORE,0.058440,0.676701,0.598815,0.679592,-0.026428,0.691417,0.058449


Show here, XGBregressor has best r2_score is approximately 0.703370

## FOR UNIT PRICE

In [88]:
final_list=[]
final_list2=[]
final_list3=[]
model=[LinearRegression(),DecisionTreeRegressor(),RandomForestRegressor(),xgb.XGBRegressor(),AdaBoostRegressor(),SVR(),GradientBoostingRegressor(),Lasso()]
result={}
for j in [5, 10, 15, 20,25, 30,35,39]:
        new=[]
       
        for i in model:
            
            
            i.fit(X_train,Y2_train)
            pred=i.predict(X_test)
            rmse = np.sqrt(mean_squared_error(Y2_test, pred))
            new.append(i.score(X_test, Y2_test))
            # scores=(cross_val_score(i,X_train,Y1_train,scoring="neg_mean_squared_error"))
        final_list.append(new)

In [89]:
for i in model:
    scores=(cross_val_score(i,X_train,Y2_train,scoring="neg_mean_squared_error"))
    print(scores.mean(),end="\t")

-17.979706575711454	-10.194947550607287	-10.4951512762085	-10.583713289372914	-11.225875921168509	-18.609242156254624	-9.992701200467591	-17.97953296923253	

In [90]:
pd.DataFrame(final_list,index=['5', '10', '15', '20', '25', '30','35','39'], columns=['Linear Regression','Decision Tree Regressor','Random Forest Regressor','XGBRegressor()','AdaBoostRegressor','svr','gradient boost','Lasso'])

,Linear Regression,Decision Tree Regressor,Random Forest Regressor,XGBRegressor(),AdaBoostRegressor,svr,gradient boost,Lasso
5,0.06816,0.828668,0.838492,0.87597,0.817502,-0.007683,0.848823,0.068119
10,0.06816,0.823061,0.840220,0.87597,0.699665,-0.007683,0.848793,0.068119
15,0.06816,0.821880,0.841624,0.87597,0.784256,-0.007683,0.848929,0.068119
20,0.06816,0.821523,0.842474,0.87597,0.733757,-0.007683,0.849286,0.068119
25,0.06816,0.822434,0.840546,0.87597,0.675314,-0.007683,0.847053,0.068119
30,0.06816,0.828896,0.837855,0.87597,0.752435,-0.007683,0.846961,0.068119
35,0.06816,0.828055,0.849262,0.87597,0.802025,-0.007683,0.849021,0.068119
39,0.06816,0.823229,0.841225,0.87597,0.654105,-0.007683,0.847125,0.068119


In [91]:
all=[]
mse=[]
rms=[]
r2score=[]

for i in model:
    i.fit(X_train,Y2_train)
    pred=i.predict(X_test)
    mse.append(mean_squared_error(Y2_test, pred))
    rms.append(np.sqrt(mean_squared_error(Y2_test, pred)))
    r2score.append(r2_score(Y2_test, pred))
all.append(mse)
all.append(rms)
all.append(r2score)

In [92]:
pd.DataFrame(all,index=['MEAN SQUARED ERROR','ROOT MEAN SQUARED ERROR','R2 SCORE'], columns=['Linear Regression','Decision Tree Regressor','Random Forest Regressor','XGBRegressor','AdaBoostRegressor','svr','gradient boost','Lasso'])

,Linear Regression,Decision Tree Regressor,Random Forest Regressor,XGBRegressor,AdaBoostRegressor,svr,gradient boost,Lasso
MEAN SQUARED ERROR,7.461906,1.413052,1.369501,0.993197,2.393966,8.069237,1.218369,7.462239
ROOT MEAN SQUARED ERROR,2.731649,1.188719,1.170257,0.996593,1.547245,2.840640,1.103797,2.731710
R2 SCORE,0.068160,0.823539,0.828977,0.875970,0.701043,-0.007683,0.847851,0.068119


## FOR FREIGHT COST

In [93]:
final_list=[]
final_list2=[]
final_list3=[]
model=[LinearRegression(),DecisionTreeRegressor(),RandomForestRegressor(),xgb.XGBRegressor(),AdaBoostRegressor(),SVR(),GradientBoostingRegressor(),Lasso()]
result={}
for j in [5, 10, 15, 20,25, 30,35,39]:
        new=[]
       
        for i in model:
            
            
            i.fit(X_train,Y3_train)
            pred=i.predict(X_test)
            rmse = np.sqrt(mean_squared_error(Y3_test, pred))
            new.append(i.score(X_test, Y3_test))
            # scores=(cross_val_score(i,X_train,Y1_train,scoring="neg_mean_squared_error"))
        final_list.append(new)

In [94]:
for i in model:
    scores=(cross_val_score(i,X_train,Y3_train,scoring="neg_mean_squared_error"))
    print(scores.mean(),end="\t")

-3.9172774694465645e-21	-1505658.622148544	-1935228.0519993007	-1039088.8801264793	-8704990.963895537	-270378305.50604475	-1321670.8389587917	-0.009101576321155986	

In [95]:
pd.DataFrame(final_list,index=['5', '10', '15', '20', '25', '30','35','39'], columns=['Linear Regression','Decision Tree Regressor','Random Forest Regressor','XGBRegressor()','AdaBoostRegressor','svr','gradient boost','Lasso'])

,Linear Regression,Decision Tree Regressor,Random Forest Regressor,XGBRegressor(),AdaBoostRegressor,svr,gradient boost,Lasso
5,1.0,0.998051,0.998225,0.999082,0.958740,-0.099974,0.999642,1.0
10,1.0,0.998091,0.998235,0.999082,0.976609,-0.099974,0.999640,1.0
15,1.0,0.998049,0.997755,0.999082,0.968716,-0.099974,0.999643,1.0
20,1.0,0.998092,0.999551,0.999082,0.978068,-0.099974,0.999641,1.0
25,1.0,0.999572,0.998863,0.999082,0.973465,-0.099974,0.999642,1.0
30,1.0,0.998099,0.998151,0.999082,0.970884,-0.099974,0.999641,1.0
35,1.0,0.998064,0.998770,0.999082,0.965293,-0.099974,0.999641,1.0
39,1.0,0.998035,0.998615,0.999082,0.976251,-0.099974,0.999643,1.0


In [101]:
all=[]
mse=[]
rms=[]
r2score=[]

for i in model:
    i.fit(X_train,Y3_train)
    pred=i.predict(X_test)
    mse.append(mean_squared_error(Y3_test, pred))
    rms.append(np.sqrt(mean_squared_error(Y3_test, pred)))
    r2score.append(round(r2_score(Y3_test, pred),4))
all.append(mse)
all.append(rms)
all.append(r2score)

In [102]:
pd.DataFrame(all,index=['MEAN SQUARED ERROR','ROOT MEAN SQUARED ERROR','R2 SCORE'], columns=['Linear Regression','Decision Tree Regressor','Random Forest Regressor','XGBRegressor','AdaBoostRegressor','svr','gradient boost','Lasso'])

,Linear Regression,Decision Tree Regressor,Random Forest Regressor,XGBRegressor,AdaBoostRegressor,svr,gradient boost,Lasso
MEAN SQUARED ERROR,8.223395e-23,478212.970369,399662.867100,239371.258901,7.767820e+06,2.869244e+08,93291.178216,0.006057
ROOT MEAN SQUARED ERROR,9.068294e-12,691.529443,632.188949,489.255822,2.787081e+03,1.693884e+04,305.436046,0.077825
R2 SCORE,1.000000e+00,0.998200,0.998500,0.999100,9.702000e-01,-1.000000e-01,0.999600,1.000000
